In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

C:\MAD4AG


In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import seaborn as sns
import collections
import folium
from folium.plugins import HeatMapWithTime
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

plt.style.use('seaborn-white')




In [3]:
file_name = f'./dbs/big_geodata_se/stops.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)

In [4]:
print('The number of stops: ',len(df))
print('The number of people: ', len(df['uid'].unique()))

df_week = df[df.holiday_s != 1]
df_week = df_week[df_week.weekday_s == 1]

print('The number of stops in weekday and non-holiday: ', len(df_week))

print('The number of people: ', len(df_week['uid'].unique()))

The number of stops:  29122046
The number of people:  322919
The number of stops in weekday and non-holiday:  13493110
The number of people:  322477


In [5]:

print(df.head(100))

                                     uid           localtime        lat  \
0   00008608-f79e-414d-bf1c-25632d6bc059 2019-07-07 16:26:55  59.330000   
1   00008608-f79e-414d-bf1c-25632d6bc059 2019-07-18 00:29:02  56.050000   
2   00008608-f79e-414d-bf1c-25632d6bc059 2019-08-28 20:56:04  56.174205   
3   00008608-f79e-414d-bf1c-25632d6bc059 2019-08-29 19:22:15  56.174205   
4   00008608-f79e-414d-bf1c-25632d6bc059 2019-08-30 22:10:18  56.174205   
..                                   ...                 ...        ...   
95  00008608-f79e-414d-bf1c-25632d6bc059 2019-11-09 14:57:25  56.092351   
96  0000c837-ef82-4dfd-b2a5-00bdc8680b0b 2019-06-03 15:32:52  59.371264   
97  0000c837-ef82-4dfd-b2a5-00bdc8680b0b 2019-06-04 15:09:24  59.371264   
98  0000c837-ef82-4dfd-b2a5-00bdc8680b0b 2019-06-04 16:36:44  59.371277   
99  0000c837-ef82-4dfd-b2a5-00bdc8680b0b 2019-06-04 18:51:22  59.371277   

          lng  loc        h_s         dur  holiday_s  weekday_s  seq  
0   18.050000   40  16.43333

In [6]:
print("Data cover %s stop points from %s devices."%(len(df), df['uid'].nunique()))

Data cover 29122046 stop points from 322919 devices.


In [7]:
def user_stats(data):
    # No. of unique locations
    num_loc = data.loc[:, 'loc'].nunique()
    # No. of active days

    # No. of stays
    num_stays = len(data)
    # Description of active days
    # ## Median value of stays/active day
    # num_stays_act = data['date'].value_counts().median()
    # ## Median value of total duration of stays/active day (h)
    # dur_total_act = np.median(data.groupby('date')['dur'].sum()/60)
    ## Median value of duration/stay (min)
    dur_median = data['dur'].median()
    return pd.Series(dict(num_loc=num_loc,  num_stays=num_stays, dur_median=dur_median))
tqdm.pandas()
df_user = df.groupby('uid').progress_apply(user_stats).reset_index()

  0%|          | 0/322919 [00:00<?, ?it/s]

In [8]:
df_user.head(100)

,uid,num_loc,num_stays,dur_median
0,00008608-f79e-414d-bf1c-25632d6bc059,43.0,96.0,179.983333
1,0000c837-ef82-4dfd-b2a5-00bdc8680b0b,11.0,114.0,180.400000
2,0000cd68-c931-4e3c-96f6-7c5837f59b08,34.0,149.0,104.550000
3,0000f6ad-ffa4-4af2-9c2a-49d6dc86ec3a,12.0,231.0,179.983333
4,000115f0-937a-4716-8d8b-09b1ed54c5ce,47.0,305.0,142.416667
...,...,...,...,...
95,00149019-ded0-4649-93d0-60911423f762,21.0,79.0,181.750000
96,0014a1c9-4e79-4aaa-936a-cc7ffc279df0,32.0,144.0,189.108333
97,0014bcb6-ab4f-4582-bdca-3870bb2c8f6a,4.0,110.0,201.075000
98,0014cd8e-d31e-471c-bf2e-f539a1d442b3,6.0,43.0,183.166667


# select an individual from the data

In [9]:
individual_uid= '1676cccd-258d-4433-a3c6-4c0cfc0f8c29'
#individual_uid= '00029d26-b702-4ac4-aacc-644f99688261'
#individual_uid= '000ce923-767e-4165-88db-1e83bbef8165'
#individual_uid= '00106d02-52ca-445e-aea7-8085a65be212'

#individual_uid= '0030377b-f4d1-411a-bebe-33edd92f4774'
#individual_uid= '6626de3e-1efd-4ea2-9900-eb02d2af6231'
individual_uid= '9b0b36c9-b1c0-444e-8708-e2751915a587'
individual_uid= '1ceb2be6-1513-4333-af36-fe007d24a5bb'




individual = df[(df['uid']==individual_uid)&(df['weekday_s']==1)]


## repeat data points using starting time and stay duration

In [10]:
individual['dur_rp']= (individual['dur']/15)+1
individual['dur_rp']= individual['dur_rp'].astype(int)

individual = pd.DataFrame(np.repeat(individual.values, individual['dur_rp'].tolist(), axis=0), columns=individual.columns)



In [11]:
individual = individual.groupby(['uid','seq'],as_index = False)
individual = individual.apply(lambda x: x.reset_index(drop = True)).reset_index()
individual = individual.drop('level_0',axis = 1).rename(columns={"level_1": "stay_seq"})

In [12]:
individual['timestamp']= pd.to_datetime(individual['localtime'])+(pd.Timedelta(minutes=15)*individual['stay_seq'])

In [13]:
time_index = []
for i in individual['timestamp']:
    time_index.append(i)
#date_strings = [pd.Timestamp(d).round(freq='30T').replace(day=1,month=7).strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]
date_strings = [pd.Timestamp(d).round(freq='30T').replace(day=1,month=7).strftime('%H:%M:%S') for d in time_index]

individual['timestamp']= date_strings
individual= individual.sort_values(by=['timestamp'])


In [14]:
individual['timestamp']= pd.to_datetime(individual['timestamp'])
time_index = []
for i in individual['timestamp'].unique():
    time_index.append(i)
date_strings = [pd.Timestamp(d).strftime('%H:%M:%S') for d in time_index]

In [15]:
#formatting the data into a nested list
lat_long_list = []
for i in individual['timestamp'].unique():
    temp=[]
    for index, instance in individual[individual['timestamp'] == i].iterrows():
        temp.append([instance['lat'],instance['lng']])
    lat_long_list.append(temp)

In [17]:
[individual['lat'].mean(),individual['lng'].mean()]

[64.81511742859587, 20.875144746591687]

In [18]:


#Choosing the map type
m = folium.Map(location=[individual['lat'].mode(),individual['lng'].mode()], zoom_start=10, tiles="cartodbdark_matter")

#Plot it on the map
HeatMapWithTime(lat_long_list,radius=7,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(m)

# Display the map
m


In [280]:
m.save('./figures/heat_map'+individual_uid+'.html')


In [12]:
# group same location visits into a single point and write timestamps to a list

individual_gr = individual.groupby(['lat','lng'])['timestamp'].apply(list).to_frame('timestamp_sum')
individual_gr.reset_index(inplace=True)

top_timestamp = []

map = folium.Map(location=[individual['lat'].mode(),individual['lng'].mode()], zoom_start=13)#, crs='EPSG4326')

for point in range(0, len(individual_gr['lat'])):
    counter = collections.Counter(individual_gr.loc[point]['timestamp_sum'])
    top_timestamp.append(counter.most_common(10))
    folium.Circle([individual_gr.loc[point]['lat'], individual_gr.loc[point]['lng']], fill_color="orange", fill_opacity=0.4, popup=top_timestamp[point]).add_to(map)
map


In [282]:
map.save('./figures/individual_'+individual_uid+'.html')


# DBSCAN

In [13]:
from sklearn.cluster import DBSCAN


In [14]:
# represent points consistently as (lat, lon)
coords = individual_gr[['lat', 'lng']].values

# define the number of kilometers in one radian
kms_per_radian = 6371.0088

# define epsilon as 1.5 kilometers, converted to radians for use by haversine
epsilon = 0.1 / kms_per_radian

In [15]:
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_

individual_gr.loc[:, 'cluster'] = cluster_labels



In [16]:
top_timestamp = []

#colors = ['red','blue','orange','green','darkgreen','lightgreen','darkblue','lightblue','purple','darkpurple','pink','cadetblue']

colors = ['cadetblue','red','blue','orange','green','purple','darkgreen','darkblue']

map = folium.Map(location=[individual['lat'].mode(),individual['lng'].mode()], zoom_start=13)#, crs='EPSG4326')

for point in range(0, len(individual_gr['lat'])):

    counter = collections.Counter(individual_gr.loc[point]['timestamp_sum'])
    top_timestamp.append(counter.most_common(10))
    folium.Circle([individual_gr.loc[point]['lat'], individual_gr.loc[point]['lng']], fill_color=colors[individual_gr.loc[point]['cluster']%8], fill_opacity=0.4, color=colors[individual_gr.loc[point]['cluster']%8], popup=top_timestamp[point]).add_to(map)

map


In [287]:
map.save('./figures/individual_cluster_'+individual_uid+'.html')


## mean point of clusters and snap to the closest building

In [17]:
individual= pd.merge(individual, individual_gr[['lat','lng','cluster']], on=['lat','lng'])

In [18]:
cluster_mean = [individual.groupby(['cluster'])['lat'].mean(),individual_gr.groupby(['cluster'])['lng'].mean()]


cluster_mean = pd.DataFrame(cluster_mean)

cluster_mean = cluster_mean.T




In [19]:
top_timestamp = []


colors = ['cadetblue','red','blue','orange','green','purple','darkgreen','darkblue']

map = folium.Map(location=[individual['lat'].mode(),individual['lng'].mode()], zoom_start=13)#, crs='EPSG4326')

for point in range(0, len(individual_gr['lat'])):

    counter = collections.Counter(individual_gr.loc[point]['timestamp_sum'])
    top_timestamp.append(counter.most_common(10))
    folium.Circle([individual_gr.loc[point]['lat'], individual_gr.loc[point]['lng']], fill_color=colors[individual_gr.loc[point]['cluster']%8], fill_opacity=0.4, color=colors[individual_gr.loc[point]['cluster']%8], popup=top_timestamp[point]).add_to(map)

for point in range(0, len(cluster_mean['lat'])):

    folium.Marker([cluster_mean.loc[point]['lat'], cluster_mean.loc[point]['lng']], tooltip="Cluster %s center" % str(point)).add_to(map)

map

In [20]:
map.save('./figures/individual_longest_commute_'+individual_uid+'.html')


In [333]:
from shapely.geometry import Point, Polygon
p = Point(0,0)
poly = Polygon([[1,0], [1,10], [10, 0]])
print(p.distance(poly))

1.0


In [344]:
import geopandas as gpd
from shapely.ops import snap, nearest_points


snapdist = 100 #The distance from point to polygon to search and snap

building_poly = gpd.read_file(f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/Geodatabase.gdb', layer= 'Buildings_dataset_raw')
print(building_poly.crs)

epsg:3006


In [351]:
building_poly.head()

,OBJEKT_ID,OBJEKT_VER,DETALJTYP,ADAT,INSAM_LAGE,XYFEL,NAMN1,NAMN2,NAMN3,HUVUDBYGGN,...,ANDAMAL_6,ANDAMAL_7,ANDAMAL_8,ANDAMAL_9,ANDAMAL_10,HUSNR,Shape_Length,Shape_Area,TARGET_FID,geometry
0,a9d891a3-c8b1-4c06-b4b1-98bb87a74227,1,HUS,2013-04-05 15:46,4,0,,,,,...,0,0,0,0,0,0,40.000489,96.002910,NaN,"MULTIPOLYGON (((484523.880 6443262.612, 484517..."
1,c7480e52-7a02-4b6c-a4b4-4cf8c23b1cee,2,HUS,2014-08-05 09:29,4,0,,,,,...,0,0,0,0,0,0,44.269637,113.079799,NaN,"MULTIPOLYGON (((512324.935 6410856.541, 512332..."
2,dddf8f86-11c6-4523-9364-819a86490a33,2,HUS,2015-06-16 13:45,4,0,,,,,...,0,0,0,0,0,0,39.999404,95.997242,NaN,"MULTIPOLYGON (((479067.426 6460115.242, 479059..."
3,7ece0648-b37b-4415-8490-ecbaaefd2b5f,1,HUS,2015-08-10 15:52,4,0,,,,,...,0,0,0,0,0,0,40.000000,96.000000,NaN,"MULTIPOLYGON (((506869.301 6463163.995, 506869..."
4,ec75acda-ce48-4abb-8dc1-9a315deec9af,1,HUS,2015-09-01 08:19,4,0,,,,,...,0,0,0,0,0,0,40.000000,96.000000,NaN,"MULTIPOLYGON (((506837.407 6463046.775, 506837..."


In [352]:
building_point = gpd.read_file(f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/Geodatabase.gdb', layer= 'Buildings_SpatJ_SU_1km_DESO_poi_wgs84')


KeyboardInterrupt: 

In [ ]:
#https://gis.stackexchange.com/questions/426161/snap-points-to-polygon-lines-in-python-shapely

#poly = gpd.read_file(r"/home/bera/Desktop/GIStest/1400_buildings.geojson")
#point = gpd.read_file(r"/home/bera/Desktop/GIStest/500_points.geojson")
#snapdist = 100 #The distance from point to polygon to search and snap
#output = r"/home/bera/Desktop/GIStest/500_points_snapped.geojson"


#To measure distances in meters, the data needs to be in a projected crs with meters as units.
#  My data is in Sweden so I choose EPSG:3006
originalcrs = point.crs
poly = poly.to_crs(3006)
point = point.to_crs(3006)

#Create unique ids
poly["polyid"] = range(poly.shape[0])
point["pointid"] = range(point.shape[0])

#For each point find the closest polygon(s)
#  If there are more than one polygon within snapdistance, the points will be duplicated
poly["polygeom"] = poly.geometry #Save the geometry, or it is lost in sjoin_nearest
sj = gpd.sjoin_nearest(left_df=point, right_df=poly, how="left", max_distance=snapdist)

#Measure distances. If there is no polygon within snapdistance, return None
sj["distance"] = sj.apply(lambda x: x.geometry.distance(x.polygeom) if x.polygeom is not None else None, axis=1)

#Sort by distance and drop any duplicated points
sj = sj.sort_values(by=["pointid","distance"], ascending=True, na_position="last")
sj = sj.drop_duplicates(subset="pointid", keep="first")

print(sj[["pointid","polyid","distance"]].head(2))
#    pointid  polyid  distance
# 0        0  1422.0  47.02335
# 1        1     NaN       NaN

#So point 0 should snap to polygon 1422 if the distance < snapdist.
#Point 1 should not snap. Any other points with 0 distance (they are already intersecting a polygon) should
#  stay unchanged

#Create/find the point on the polygon to snap to
sj["nearestpoint"] = sj.apply(lambda x: nearest_points(x.geometry, x.polygeom)[1] if (x.polygeom is not None or x.distance==0) else None, axis=1)
print(sj.head(2))
#    id                        geometry  ...  distance                    nearestpoint
# 0   0  POINT (680677.291 6577536.845)  ...  47.02335  POINT (680703.749 6577497.971)
# 1   1  POINT (682069.722 6590459.711)  ...       NaN                            None

sj["geometry"] = sj.apply(lambda x: snap(x.geometry, x.nearestpoint, snapdist)
                          if x.nearestpoint is not None else x.geometry, axis=1) #Snap
#Export the result
sj = sj.to_crs(originalcrs)
sj = sj[[c for c in sj.columns if c in point.columns]] #Drop processing columns
sj.to_file(output)
